# Bader charge analysis

## Visualizing the WorkGraph Builder



In [1]:
from workgraph_collections.ase.espresso.bader import bader_workgraph
from aiida import load_profile
load_profile()

node = bader_workgraph.node()
node.to_html()

## Visualizing the WorkGraph

In [2]:
from workgraph_collections.ase.espresso.bader import bader_workgraph

wg = bader_workgraph()
wg.to_html()

## Example: Silicon

### Prepare the inputs and submit the workflow


In [5]:
from ase.build import molecule
from aiida import load_profile

from workgraph_collections.ase.espresso.bader import bader_workgraph

load_profile()

atoms = molecule('H2O', vacuum=3.0)
atoms.pbc = True


metadata = {
    "options": {
        'prepend_text' : """eval "$(conda shell.posix hook)"
        conda activate aiida
        export OMP_NUM_THREADS=1
        """,
    }
}
pseudopotentials = {"H": "H.pbe-kjpaw_psl.1.0.0.UPF", 
                    "O": "O.pbe-n-kjpaw_psl.1.0.0.UPF"}
pseudo_dir = "/home/xing/data/ase/espresso_pseudo/kjpaw"
# pseudo_dir = "/home/wang_x3/datas/pseudos/psl"
scf_input_data = {
    "control": {"calculation": "scf",
                },
    "system": {"ecutwfc": 30, "ecutrho": 240,
               "occupations": "smearing",
                "degauss": 0.01,
                "smearing": "cold"
                },
}

inputs = {
    "scf": {
        "input_data": scf_input_data,
        "kpts": (1, 1, 1),
        "metadata": metadata
    },
    "pp_valence": {
        "metadata": metadata
    },
    "pp_all": {
        "metadata": metadata
    },
    "bader": {
        "metadata": metadata
    }
}
#------------------------- Set the inputs -------------------------
wg = bader_workgraph(atoms=atoms,
                    pw_command="mpirun -np 1 pw.x",
                    pp_command="mpirun -np 1 pp.x",
                    bader_command="bader",
                    computer="localhost",
                    pseudopotentials=pseudopotentials,
                    pseudo_dir=pseudo_dir,
                    inputs=inputs)
#------------------------- Submit the calculation -------------------
# wg.run()
wg.submit(wait=True, timeout=200)
#------------------------- Print the output -------------------------
charges = wg.nodes['bader'].outputs["result"].value.value
print("Bader charges:")
print("Index    Symbol    Charge")
for i, charge in enumerate(charges):
    print(f"{i:5d}    {atoms.get_chemical_symbols()[i]:5s}    {charge:5.3f}")


WorkGraph node created, PK: 53900
Bader charges:
Index    Symbol    Charge
    0    O        7.148
    1    H        0.426
    2    H        0.426
